In [1]:
import os
import sys

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import json

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
import torch
import pandas as pd
from PIL import Image

from core.captioner import ClipCapCaptioner, DeCapCaptioner
from core.dataset import ScanNet
from core.integrate import FeatureFusionScalableTSDFVolume
from core.labeler import CLIPTextQuerier, KMeansLabeler

In [3]:
dataset = ScanNet('/home/quanta/Datasets/ScanNet/')
nyu40_color = dataset.nyu40id_to_color
nyu40_class = [dataset.nyu40_id_to_class[i] for i in range(41)]

In [4]:
scene = "scannet_scene0000_00"
load_dir = "/home/quanta/Experiments/feature-instance-fusion/" + scene + "/"

In [5]:
json_pth = os.path.abspath("../../config/views/" + scene + ".json")
with open(json_pth, "r") as f:
    views = json.load(f)['views']

In [6]:
verts = np.load(load_dir + "tsdf/verts.npy")
faces = np.load(load_dir + "tsdf/faces.npy")

In [7]:
tsdf_device = "cuda:1"
tsdf_volume = FeatureFusionScalableTSDFVolume(
    voxel_size=0.015,
    sdf_trunc=0.075,
    margin=0.08,
    device="cuda:1",
)
tsdf_volume.load(load_dir + "tsdf/tsdf_volume_unpruned.pt")
tsdf_volume.load_feats(load_dir + 'tsdf_feature_gt_semantic/feats.pt')

In [8]:
gt_labels = np.argmax(tsdf_volume.extract_feat_on_grid(verts=verts)[0], axis=1)

In [9]:
gt_counts = np.bincount(gt_labels, minlength=41)

In [12]:
tsdf_device = "cuda:1"
tsdf_volume = FeatureFusionScalableTSDFVolume(
    voxel_size=0.015,
    sdf_trunc=0.075,
    margin=0.08,
    device="cuda:1",
)
tsdf_volume.load(load_dir + "tsdf/tsdf_volume.pt")
tsdf_volume.load_feats(load_dir + "tsdf_feature_conceptfusion/feats.pt")

In [13]:
conceptfusion_feats = tsdf_volume.extract_feat_on_grid(verts=verts, device='cpu')[0]

In [14]:
del tsdf_volume
torch.cuda.empty_cache()

In [15]:
decap = DeCapCaptioner(
    decap_ckpt="/home/quanta/Models/decap/coco_prefix-009.pt",
    device=tsdf_device,
)
decap.load_model()

## Random choice

In [17]:
repeat = 10
random_feat_decode = []
for i in range(41):
    choices = np.argwhere(gt_labels == i).reshape(-1)

    if len(choices) == 0:
        random_feat_decode.append([""] * repeat)
        continue

    decoded_sents = []

    for i in range(repeat):
        idx = np.random.choice(choices)
        feat = torch.from_numpy(conceptfusion_feats[idx, :])
        text = decap.get_caption(feat)
        decoded_sents.append(text)

    random_feat_decode.append(decoded_sents)

In [19]:
random_decode = pd.DataFrame(np.array(random_feat_decode))

In [22]:
random_decode

,0,1,2,3,4,5,6,7,8,9
0,blurry photograph of man in dark suit and tie ...,blurry photograph of a person 's lap top and b...,picture of a dark - haired woman 's tie at nig...,"picture of a woman with "" "" "" "" "" "" "" "" "" "" "" ...",picture of a dark - haired person 's lap top i...,female male in dark suit in corner of zoo .,"blurry photograph of person 's lap top , with ...",picture of a female male cat with his hair in ...,blurry photograph of person in dark blue sky a...,blurry photograph of small tan cat in room wit...
1,a lone person 's hair in room with book rack a...,blurry photograph of small cat in dark room wi...,blurry photograph of a person 's lap top and m...,blurry photograph of person 's lap top and rem...,picture of a dark - haired suit case and remot...,picture of a dark - style door and chair in ho...,"blurry photograph of a person 's bed , chair ,...",blurry photograph of person 's lap top in the ...,blurry photograph of a person 's lap top and t...,picture of a tan - haired person 's lap top an...
2,blurry photograph of a toddler dog 's lap top ...,blurry photograph of a person 's beard and his...,picture of a female dog in dark blue counter .,a blurry cat 's lap top and dog in room .,blurry photograph of cat 's hair in garage,blurry photograph of baby dog 's lap at home .,blurry photograph of child 's lap top and dog ...,picture of a person with dark hair and beard i...,"picture of a dark haired person 's lap top , w...",small black and white person with a baby dog '...
3,"blurry photograph of a dark door with tv , com...","picture of dark - haired person 's lap top , i...",picture of single dark colored person 's tie a...,picture of male cat in suit and tie in room .,picture of a woman 's hand in dark colored str...,picture of baby cat in dark room with tie on t...,picture of dark haired person 's tie in living...,picture of a woman 's lap top and tan clock in...,"picture of a person with dark hair , "" "" "" "" ""...",picture of a dark - haired woman 's lap top an...
4,blurry photograph of a woman 's lap top and be...,this black cat 's hand 's bed with her hair an...,blurry photograph of baby cat 's lap and suitc...,blurry photograph of a toddler bed with his la...,picture of person in dark colored suit with ey...,picture of person with long hair and dark stri...,picture of baby cat 's bed with leather shoes ...,this dark - colored person 's lap top in corne...,blurry photograph of a toddler dog 's lap top ...,this dark colored cat 's bed with his arm and ...
5,,,,,,,,,,
6,picture of a dark colored leather dog 's lap t...,a dark haired cat 's couch in living room area .,a dark haired cat 's couch in living room area .,blurry photograph of a person 's lap top and b...,blurry photograph of baby dog 's lap and neck ...,a dark haired cat 's couch in living room area .,blurry photograph of a person 's lap top and b...,baby dog 's lap at camera in dark area .,blurry photograph of a person 's lap top and b...,a dark haired cat 's couch in living room area .
7,blurry image of person with dark hair and dog ...,black and white photograph of person 's hair w...,picture of a woman 's hair with dark tie and l...,picture of a person in dark suit and baby cat ...,picture of a white cat with a drink in microwa...,blurry photograph of coffee table with a perso...,blurry photograph of a coffee counter with his...,blurry photograph of a woman 's lap top and br...,picture of a female dog in dark suit case with...,picture of person in dark colored suit at came...
8,blurry photograph of a person in dark suit and...,blurry photograph of a person in dark suit and...,blurry photograph of a person 's hair in dark ...,"picture of a dark - colored door "" "" "" "" "" "" ""...",blurry photograph of a woman with his lap top ...,"picture of a dark - colored door "" "" "" "" "" "" ""...",picture of a woman 's tan and striped clock in...,picture of a dark haired woman in suit and tie...,picture of a dark colored cat with his arm in .

## Mean decode

In [24]:
mean_decode = []
for i in range(41):
    choices = np.argwhere(gt_labels == i).reshape(-1)

    if choices.shape[0] == 0:
        mean_decode.append("")
        continue

    feat = torch.from_numpy(conceptfusion_feats[choices, :]).mean(dim=0)
    text = decap.get_caption(feat)
    mean_decode.append(text)

In [25]:
mean_decode = pd.DataFrame(mean_decode)

In [26]:
mean_decode

,0
0,picture of a dark haired person 's hair in dar...
1,picture of a dark - haired woman 's lap top an...
2,blurry photograph of a persons dog 's lap top ...
3,picture of a dark - haired woman 's lap top an...
4,blurry photograph of a person 's lap top and b...
5,
6,blurry photograph of a person 's lap top and b...
7,blurry photograph of a person 's lap top and t...
8,picture of a dark - haired woman 's lap top an...
9,blurry photograph of person in dark blue sky a...


In [27]:
feat_count = pd.DataFrame(gt_counts)

In [28]:
class_name = pd.DataFrame(nyu40_class)

In [30]:
save_dir = '../03_caption/04_conceptfusion_3D_caption/'
os.makedirs(save_dir, exist_ok=True)

In [31]:
random_decode.to_csv(save_dir + 'random_decode.csv', index=False, header=False)
mean_decode.to_csv(save_dir + 'mean_decode.csv', index=False, header=False)
feat_count.to_csv(save_dir + 'semantic_count.csv', index=False, header=False)
class_name.to_csv(save_dir + 'semantic_name.csv', index=False, header=False)